# Jupyter Notebook for the evaluation of nucleon PDFs from Distillation correlation functions

## Initialization

In [1]:
import numpy as np
import scipy.linalg as la
import scipy.optimize as scipyOpt
import math, itertools as it
import matplotlib as mpl ; import matplotlib.pyplot as plt ; plt.ion()
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetiva']})
rc('text', usetex=True)

print ("Done")

Done


## Directories, Parameters, Constants

### Generic definitions

In [29]:
def MomTag(zMom):
    mSgn = lambda i: ("+" if i > 0 else "") + str(i)
    return 'pz.%s'%(mSgn(zMom))

def phMomTpl(phTag,zMom):
    return (phTag,MomTag(zMom))

def momFileTag(mVec):
    mFTag = 'momXYZ.%d.%d.%d' % (mVec[0],mVec[1],mVec[2])
    return mFTag
#-----------------------------------


# Parameters for unphased correlators
phInfo = 'unphased'

# Note: Every combination of momentum, t0 and t_sink is considered a 'dataset'
ZmomVal   = {phInfo: [0,1,2,3,-1,-2,-3]} # Available momenta values
TsinkList = {phInfo: [4,6,8,10,12,14]}   # Tsink Values
T0List    = {phInfo: [0,16,32,48]}       # T0 Values

# Insertion gamma matrices
gMatList = ['gt',
            'gxg5','gyg5','gzg5','gtg5',
            'gxgy','gxgz','gxgt','gygz','gygt','gzgt']

# (Maximum) Number of configs
Ncfg_all  = 349

# Insertion operators list, that's a common file among all momenta
insOpFile_Dict = {phInfo: '../pPDF/insert_gamma_row.DA.list'}


# Base Directory of 3-point correlation functions
corr3pt_dir = {phInfo: '../pPDF/3pt_data_%s'%(phInfo)}


momLabels_Dict = {phInfo: []}
momValue_Dict  = {}
Ncfg_Dict = {}
existsDataSet = {}
for zmom in ZmomVal[phInfo]:
    mTag = MomTag(zmom)
    pmTpl = phMomTpl(phInfo,zmom)
    momLabels_Dict[phInfo].append(mTag)
    momValue_Dict[mTag] = np.array([0,0,zmom])
    
    for t0 in T0List[phInfo]:
        for tsnk in TsinkList[phInfo]:
            dTpl = (phInfo,mTag,t0,tsnk)
            Ncfg_Dict[dTpl] = Ncfg_all  # All datasets have the same number of configs
            existsDataSet[dTpl] = True  # All possible datasets exist
#----------------------------------------------

print('Momentum Labels:\n', momLabels_Dict)
print('\n')
print('Momentum Values:\n', momValue_Dict)

print('\nDone\n')

Momentum Labels:
 {'unphased': ['pz.0', 'pz.+1', 'pz.+2', 'pz.+3', 'pz.-1', 'pz.-2', 'pz.-3']}


Momentum Values:
 {'pz.0': array([0, 0, 0]), 'pz.+1': array([0, 0, 1]), 'pz.+2': array([0, 0, 2]), 'pz.+3': array([0, 0, 3]), 'pz.-1': array([ 0,  0, -1]), 'pz.-2': array([ 0,  0, -2]), 'pz.-3': array([ 0,  0, -3])}


Insertion operator List:
 {'name': ['b_b0xDA__J0_A1pP', 'a_a1xDA__J1_T1pM', 'pion_pion_2xDA__J0_A1mM', 'b_b1xDA__J1_T1pP', 'rho_rho_2xDA__J1_T1mP', 'a_a1xDA__J1_T1pM', 'a_a1xDA__J1_T1pM', 'b_b1xDA__J1_T1pP', 'b_b1xDA__J1_T1pP', 'rho_rho_2xDA__J1_T1mP', 'rho_rho_2xDA__J1_T1mP'], 'row': [1, 2, 1, 2, 2, 1, 3, 1, 3, 1, 3]}

Done



### Read/define source-insertion-sink operators

In [32]:

# Interpolating operators  for 3-point functions         
nRowSrc3pt = {'single': 2}
nRowSnk3pt = {'single': 2}
srcOpList3pt = {'single': {}}
snkOpList3pt = {'single': {}}
for zmom in ZmomVal[phInfo]:
    pmTpl = phMomTpl(phInfo,zmom)
    if zmom == 0:
        srcOpList3pt['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
        snkOpList3pt['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
    else:
        srcOpList3pt['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        snkOpList3pt['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        
        
# FIXME: Interpolating operators for 2-point functions        

#----------------------------------------------


# Read insertion operators
insOptrList = {}
insOpFname = insOpFile_Dict[phInfo]
insOptrList[phInfo] = {'name': [], 'row': []}
with open(insOpFname) as fp:
    line = fp.readlines()
    for t in line:
        insOptrList[phInfo]['name'].append(t.split()[0])
        insOptrList[phInfo]['row'].append(int(t.split()[1]))

# Create dictionary to map actual gamma matrices to operator names and rows
gMat = {phInfo: {'gt'  : ('b_b0xDA__J0_A1pP', 1),
                 'gxg5': ('a_a1xDA__J1_T1pM', 1),
                 'gyg5': ('a_a1xDA__J1_T1pM', 3),
                 'gzg5': ('a_a1xDA__J1_T1pM', 2),
                 'gtg5': ('pion_pion_2xDA__J0_A1mM', 1),
                 'gxgy': ('b_b1xDA__J1_T1pP', 2),
                 'gxgz': ('b_b1xDA__J1_T1pP', 3),
                 'gxgt': ('rho_rho_2xDA__J1_T1mP', 3),
                 'gygz': ('b_b1xDA__J1_T1pP', 1),
                 'gygt': ('rho_rho_2xDA__J1_T1mP', 1),
                 'gzgt': ('rho_rho_2xDA__J1_T1mP', 2)}}
#----------------------------------------------

print('Insertion operator List:\n', insOptrList[phInfo])

print('\nDone\n')

Insertion operator List:
 {'name': ['b_b0xDA__J0_A1pP', 'a_a1xDA__J1_T1pM', 'pion_pion_2xDA__J0_A1mM', 'b_b1xDA__J1_T1pP', 'rho_rho_2xDA__J1_T1mP', 'a_a1xDA__J1_T1pM', 'a_a1xDA__J1_T1pM', 'b_b1xDA__J1_T1pP', 'b_b1xDA__J1_T1pP', 'rho_rho_2xDA__J1_T1mP', 'rho_rho_2xDA__J1_T1mP'], 'row': [1, 2, 1, 2, 2, 1, 3, 1, 3, 1, 3]}

Done



### Ensemble-related definitions

In [30]:
# Ensemble parameters
L = 32
T = 64

binsize=1
#------------------------------------------------

# Nucleon, pion mass
alat_fm = 0.094
alat_iGeV = alat_fm / 0.1973
mpi_GeV = 0.350
mN_GeV = 1.123
amN = mN_GeV * alat_iGeV

# Dispersion relation
Pi = np.pi
def dispRel(am,k):
    ap = 2*Pi/L * k
    return np.sqrt(am**2 + np.dot(ap,ap))